For each year we will precalculate levels of gas leaks per person for each geoid. Then we will group geoids by levels and create geolayers. So, each layer will have a group of geoid with ~ same level of gas leaks per person

In [90]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np
import re

mapbox_access_token = "pk.eyJ1IjoibWlzaGtpY2UiLCJhIjoiY2s5MG94bWRoMDQxdjNmcHI1aWI1YnFkYyJ9.eFsHqEMYY7qxa0Pb9USCtQ"

In [91]:
dfg = gpd.read_file('C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/2010 Census Tracts with geoid.geojson')
dfg['geoid'] = dfg.block_fips.str[:11]
dfg = dfg[['geoid', 'ntaname', 'boro_name', 'shape_area', 'geometry']]
dfg.columns = map(str.lower, dfg.columns)
dfg = dfg.applymap(lambda s:s.lower() if type(s) == str else s)

In [92]:
dfg.head()

,geoid,ntaname,boro_name,shape_area,geometry
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ..."
1,36061009800,turtle bay-east midtown,manhattan,1906016.35002,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ..."
2,36061010000,turtle bay-east midtown,manhattan,1860938.37721,"MULTIPOLYGON (((-73.96802 40.75958, -73.96848 ..."
3,36061010200,midtown-midtown south,manhattan,1860992.70485,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ..."
4,36061010400,midtown-midtown south,manhattan,1864600.37503,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ..."


In [93]:
dfg[dfg['ntaname'].str.contains('cemetery')].head()

,geoid,ntaname,boro_name,shape_area,geometry
774,36061000100,park-cemetery-etc-manhattan,manhattan,1844421.23744,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
889,36047017500,park-cemetery-etc-brooklyn,brooklyn,21101598.5644,"MULTIPOLYGON (((-73.99238 40.65890, -73.99019 ..."
1001,36081062400,park-cemetery-etc-queens,queens,5374590.6882,"MULTIPOLYGON (((-73.74125 40.68746, -73.73760 ..."
1226,36047015400,park-cemetery-etc-brooklyn,brooklyn,9969049.01193,"MULTIPOLYGON (((-74.02053 40.61806, -74.01984 ..."
1289,36047070203,park-cemetery-etc-brooklyn,brooklyn,43216571.4155,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ..."


In [94]:
full_df = pd.read_csv('C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\important_(used_in_app)\\Merged_asc_fdny_data.csv')

In [95]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [96]:
full_df = full_df.dropna()

In [97]:

full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11577 entries, 0 to 11606
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   %housh_grandp_resp_for_grandch  11577 non-null  float64
 1   avg_houshold_size               11577 non-null  float64
 2   avg_year_built                  11577 non-null  float64
 3   gas_leaks                       11577 non-null  int64  
 4   gas_leaks_per_person            11577 non-null  float64
 5   geoid                           11577 non-null  int64  
 6   incident_year                   11577 non-null  int64  
 7   lonely_housholder%              11577 non-null  float64
 8   lonely_housholder_over65%       11577 non-null  float64
 9   mean_houshold_income            11577 non-null  float64
 10  median_age                      11577 non-null  float64
 11  nonfamily_housholds%            11577 non-null  float64
 12  not_us_citizen%                 

In [98]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_year_built,gas_leaks,gas_leaks_per_person,geoid,incident_year,lonely_housholder%,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%
count,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2.909000e+03,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000
mean,2.216174,2.542169,1949.813021,23.303541,0.008117,3.604870e+10,2015.775524,35.098592,10.982289,82582.104160,35.928292,43.406272,14.571798,1811.629770,10.415607,9.839464
std,3.077874,0.530485,13.692450,16.265898,0.032271,2.513603e+07,1.557740,13.148335,7.025833,57297.932858,6.615947,16.327671,7.396101,1242.520399,6.536692,7.717637
min,0.000000,1.030000,1922.479141,1.000000,0.003310,3.600500e+10,2013.000000,0.000000,0.000000,11652.000000,15.700000,0.000000,0.000000,15.000000,0.000000,0.000000
25%,0.000000,2.200000,1940.207246,12.000000,0.003938,3.604702e+10,2015.000000,26.084837,6.666667,45543.000000,31.700000,31.300161,9.385010,1031.000000,5.700000,5.200000
50%,1.436389,2.550000,1949.193487,19.000000,0.004934,3.604711e+10,2016.000000,33.333333,9.851301,65644.000000,34.900000,41.434092,13.337893,1544.000000,9.300000,8.400000
75%,3.407683,2.900000,1957.325547,30.000000,0.007033,3.606102e+10,2017.000000,42.331288,13.902681,98881.000000,39.000000,53.185956,18.850777,2269.000000,13.900000,12.500000
max,74.647887,5.050000,2003.839184,164.000000,1.133333,3.608503e+10,2018.000000,96.814672,82.526316,458625.000000,84.300000,100.000000,56.237817,13033.000000,78.300000,83.900000


<h3>Add bins column</h3>

In [99]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10",
    "park_cemetery"
]
bins_leaks = [
    "0-10",
    "11-20",
    "21-30",
    "31-40",
    "41-50",
    "51-70",
    "71-200",
    "park_cemetery"
]

In [100]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [101]:
full_df['bins_leaks']=full_df['gas_leaks'].apply(lambda x: bins_leaks[0] if float(x) <=10 else bins_leaks[1] if float(x) <=20 else bins_leaks[2] if float(x) <=30 else bins_leaks[3] if float(x) <=40 else bins_leaks[4] if float(x) <=50 else bins_leaks[5] if float(x) <=70 else bins_leaks[6] )

In [102]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [103]:
merged = dfg.merge(full_df, on='geoid' )

In [104]:
merged.head()

,geoid,ntaname,boro_name,shape_area,geometry,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_year_built,gas_leaks,gas_leaks_per_person,...,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%,bins_leaks_per_person,bins_leaks
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.430416,2.94,1941.950980,1,0.000486,...,1.434720,75868.0,32.6,29.842181,16.431697,918,11.3,24.1,0-0.0015,0-10
1,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.756430,2.96,1942.820815,9,0.004587,...,1.815431,68348.0,29.5,28.895613,18.858308,932,9.2,29.1,0.00451-0.006,0-10
2,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.636943,2.96,1945.288901,10,0.005376,...,3.343949,69225.0,30.0,25.796178,17.580645,919,11.0,31.7,0.00451-0.006,0-10
3,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.650407,2.80,1948.739501,6,0.003480,...,8.292683,62529.0,31.7,33.333333,16.531323,881,11.6,30.2,0.0031-0.0045,0-10
4,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.000000,2.72,1944.240783,5,0.002846,...,7.894737,69282.0,32.7,33.436533,15.651679,868,9.8,25.6,0.00151-0.003,0-10


In [105]:
for index, row in merged.iterrows():
    if 'cemete' in row['ntaname']:
        merged.loc[index, 'bins_leaks_per_person'] = bins_leaks_per_person[7]
        merged.loc[index, 'bins_leaks'] = bins_leaks[7]

In [125]:
years = [2013 + x for x in range(6)]
for year in years:
    for bin in bins_leaks_per_person:
        print(bin)
        geo_layer = merged[(merged['bins_leaks_per_person'] == bin) & (merged['incident_year'] == year)] # geo

        try:
            geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/rel/{}.geojson".format(str(year)+'_'+bin), driver='GeoJSON')
        except:
            print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [106]:
years = [2013 + x for x in range(6)]
for year in years:
    for bin in bins_leaks:
        print(bin)
        geo_layer = merged[(merged['bins_leaks'] == bin) & (merged['incident_year'] == year)] # geo

        try:
            geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/abs/{}.geojson".format(str(year)+'_'+bin), driver='GeoJSON')
        except:
            print('err')

0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery
0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery
0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery
0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery
0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery
0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery


In [57]:
full_df = pd.read_csv('https://raw.githubusercontent.com/MarinaOrzechowski/GasLeakConEd/timeline_branch/data/processed/important_(used_in_app)/Merged_asc_fdny_data.csv')

In [58]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [59]:
full_df = full_df.dropna()

In [60]:

full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11577 entries, 0 to 11606
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   geoid                           11577 non-null  int64  
 1   incident_year                   11577 non-null  int64  
 2   gas_leaks                       11577 non-null  int64  
 3   gas_leaks_per_person            11577 non-null  float64
 4   median_age                      11577 non-null  float64
 5   avg_houshold_size               11577 non-null  float64
 6   avg_year_built                  11577 non-null  float64
 7   nonfamily_housholds%            11577 non-null  float64
 8   lonely_housholder%              11577 non-null  float64
 9   lonely_housholder_over65%       11577 non-null  float64
 10  %housh_grandp_resp_for_grandch  11577 non-null  float64
 11  unemployed%                     11577 non-null  float64
 12  mean_houshold_income            

In [61]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,geoid,incident_year,gas_leaks,gas_leaks_per_person,median_age,avg_houshold_size,avg_year_built,nonfamily_housholds%,lonely_housholder%,lonely_housholder_over65%,%housh_grandp_resp_for_grandch,unemployed%,mean_houshold_income,total_housing_units,vacant_housing_units%,not_us_citizen%
count,2.909000e+03,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000,2909.000000
mean,3.604870e+10,2015.775524,23.303541,0.008117,35.928292,2.542169,1949.813021,43.406272,35.098592,10.982289,2.216174,10.415607,82582.104160,1811.629770,9.839464,14.571798
std,2.513603e+07,1.557740,16.265898,0.032271,6.615947,0.530485,13.692450,16.327671,13.148335,7.025833,3.077874,6.536692,57297.932858,1242.520399,7.717637,7.396101
min,3.600500e+10,2013.000000,1.000000,0.003310,15.700000,1.030000,1922.479141,0.000000,0.000000,0.000000,0.000000,0.000000,11652.000000,15.000000,0.000000,0.000000
25%,3.604702e+10,2015.000000,12.000000,0.003938,31.700000,2.200000,1940.207246,31.300161,26.084837,6.666667,0.000000,5.700000,45543.000000,1031.000000,5.200000,9.385010
50%,3.604711e+10,2016.000000,19.000000,0.004934,34.900000,2.550000,1949.193487,41.434092,33.333333,9.851301,1.436389,9.300000,65644.000000,1544.000000,8.400000,13.337893
75%,3.606102e+10,2017.000000,30.000000,0.007033,39.000000,2.900000,1957.325547,53.185956,42.331288,13.902681,3.407683,13.900000,98881.000000,2269.000000,12.500000,18.850777
max,3.608503e+10,2018.000000,164.000000,1.133333,84.300000,5.050000,2003.839184,100.000000,96.814672,82.526316,74.647887,78.300000,458625.000000,13033.000000,83.900000,56.237817


<h3>Add bins column</h3>

In [31]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10",
    "park_cemetery"
]

In [32]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [34]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [35]:
merged = dfg.merge(full_df, on='geoid' )

In [36]:
merged.head()

,geoid,ntaname,boro_name,shape_area,geometry,incident_year,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,...,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%,bins_leaks_per_person
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2013,0.000486,2.0,1212.000000,32.6,...,1.434720,0.430416,11.3,60817.0,75868.0,918,75.9,24.1,16.431697,0-0.0015
1,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2014,0.004587,2.0,2006.333333,29.5,...,1.815431,0.756430,9.2,60352.0,68348.0,932,70.9,29.1,18.858308,0.00451-0.006
2,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2015,0.005376,2.1,1474.300000,30.0,...,3.343949,0.636943,11.0,52500.0,69225.0,919,68.3,31.7,17.580645,0.00451-0.006
3,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2016,0.003480,2.0,981.833333,31.7,...,8.292683,0.650407,11.6,47868.0,62529.0,881,69.8,30.2,16.531323,0.0031-0.0045
4,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2017,0.002846,2.0,754.400000,32.7,...,7.894737,0.000000,9.8,55250.0,69282.0,868,74.4,25.6,15.651679,0.00151-0.003


In [26]:
for index, row in merged.iterrows():
    if 'cemete' in row['ntaname']:
        merged.loc[index, 'bins_leaks_per_person'] = bins_leaks_per_person[7]

In [125]:
years = [2013 + x for x in range(6)]
for year in years:
    for bin in bins_leaks_per_person:
        print(bin)
        geo_layer = merged[(merged['bins_leaks_per_person'] == bin) & (merged['incident_year'] == year)] # geo

        try:
            geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format(str(year)+'_'+bin), driver='GeoJSON')
        except:
            print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [28]:

for bin in bins_leaks:
    print(bin)
    geo_layer = merged[(merged['bins_leaks'] == bin)] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [28]:

for bin in bins_leaks_per_person:
    print(bin)
    geo_layer = merged[(merged['bins_leaks_per_person'] == bin)] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


<h2> All years consolidated </h2>

In [107]:
full_df = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data_all_years.csv')

In [108]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [109]:
full_df = full_df.dropna()

In [110]:

full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2039 entries, 0 to 2050
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   %housh_grandp_resp_for_grandch  2039 non-null   float64
 1   avg_houshold_size               2039 non-null   float64
 2   avg_year_built                  2039 non-null   float64
 3   gas_leaks                       2039 non-null   int64  
 4   gas_leaks_per_person            2039 non-null   float64
 5   geoid                           2039 non-null   int64  
 6   lonely_housholder%              2039 non-null   float64
 7   lonely_housholder_over65%       2039 non-null   float64
 8   mean_houshold_income            2039 non-null   float64
 9   median_age                      2039 non-null   float64
 10  nonfamily_housholds%            2039 non-null   float64
 11  not_us_citizen%                 2039 non-null   float64
 12  total_housing_units             20

In [111]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_year_built,gas_leaks,gas_leaks_per_person,geoid,lonely_housholder%,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%
count,447.000000,447.000000,447.000000,447.000000,447.000000,4.470000e+02,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000
mean,2.191413,2.504218,1950.364055,137.881432,0.006963,3.604846e+10,36.466738,11.274226,82874.953840,35.620559,44.907075,14.229680,11245.984340,5.135567,10.268822
std,2.232453,0.528178,13.944544,93.108533,0.010147,2.442395e+07,12.879602,6.998526,59613.007944,6.589370,16.206683,6.585943,7569.770818,2.210542,7.742934
min,0.000000,1.060000,1924.695828,13.000000,0.003315,3.600500e+10,8.686441,0.000000,14351.333333,15.066667,11.334746,0.000000,158.000000,0.000000,0.000000
25%,0.263583,2.169167,1941.239085,72.000000,0.003856,3.604702e+10,27.739693,6.889609,44419.750000,31.558333,32.497435,9.604721,6529.000000,3.473019,5.591029
50%,1.585156,2.531667,1949.644414,116.000000,0.004850,3.604712e+10,34.760654,9.908357,63432.166667,34.350000,43.231489,13.202793,9716.000000,4.943529,8.881138
75%,3.501172,2.869167,1957.699342,181.500000,0.006778,3.606102e+10,42.777612,14.026841,99707.000000,38.441667,54.886360,17.920139,14406.000000,6.515282,12.615676
max,18.604651,4.963333,2001.989727,780.000000,0.164502,3.608503e+10,96.008017,76.415094,414333.166667,75.116667,99.040767,35.916113,77143.000000,14.655172,66.421707


<h3>Add bins column</h3>

In [112]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10",
    "park_cemetery"
]
bins_leaks = [
    "0-10",
    "11-20",
    "21-30",
    "31-40",
    "41-50",
    "51-70",
    "71-200",
    "park_cemetery"
]

In [113]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )
full_df['bins_leaks']=full_df['gas_leaks'].apply(lambda x: bins_leaks[0] if float(x) <=10 else bins_leaks[1] if float(x) <=20 else bins_leaks[2] if float(x) <=30 else bins_leaks[3] if float(x) <=40 else bins_leaks[4] if float(x) <=50 else bins_leaks[5] if float(x) <=70 else bins_leaks[6] )

In [114]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [115]:
merged = dfg.merge(full_df, on='geoid' )

In [116]:
merged.head()

,geoid,ntaname,boro_name,shape_area,geometry,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_year_built,gas_leaks,gas_leaks_per_person,...,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%,bins_leaks_per_person,bins_leaks
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.534759,2.845000,1944.557391,39,0.003487,...,5.347594,69233.000000,31.566667,30.837790,16.779903,5380,4.890041,27.007435,0.0031-0.0045,31-40
1,36061009800,turtle bay-east midtown,manhattan,1906016.35002,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ...",0.000000,1.700000,1953.280970,159,0.003628,...,15.507958,199916.666667,40.016667,71.019847,7.935928,31653,1.859627,19.612675,0.0031-0.0045,71-200
2,36061010000,turtle bay-east midtown,manhattan,1860938.37721,"MULTIPOLYGON (((-73.96802 40.75958, -73.96848 ...",0.000000,1.488333,1946.686894,98,0.009199,...,12.091779,154906.333333,36.216667,79.377815,14.540505,8252,2.562658,13.911779,0.0091-10,71-200
3,36061010200,midtown-midtown south,manhattan,1860992.70485,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ...",0.000000,1.898333,1976.984794,22,0.018303,...,3.030303,140282.666667,40.250000,34.343434,14.975042,1769,0.000000,66.421707,0.0091-10,21-30
4,36061010400,midtown-midtown south,manhattan,1864600.37503,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ...",0.000000,1.573333,1947.301983,38,0.006468,...,24.056095,183816.666667,53.233333,69.983819,16.612766,6277,4.323404,40.927195,0.0061-0.0075,31-40


In [117]:
for index, row in merged.iterrows():
    if 'cemete' in row['ntaname']:
        merged.loc[index, 'bins_leaks_per_person'] = bins_leaks_per_person[7]
        merged.loc[index, 'bins_leaks'] = bins_leaks[7]

In [91]:
for bin in bins_leaks_per_person:
    print(bin)
    geo_layer = merged[merged['bins_leaks_per_person'] == bin] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [118]:
for bin in bins_leaks:
    print(bin)
    geo_layer = merged[merged['bins_leaks'] == bin] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/abs/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-10
11-20
21-30
31-40
41-50
51-70
71-200
park_cemetery
